In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [52]:
torch.manual_seed(1)

## nn.Linear와 nn.Sigmoid로 Logistic Regression 구현

로지스틱 회귀의 가설식은

$ H(x) = sigmoid(Wx + b) = \frac{ 1 }{ 1 + e^{-(Wx + b)} } = \sigma(Wx + b) $

여기에서 $ (Wx + b) $ 는 Linear Regression의 가설식이다

해당 가설식은 nn.Linear()로 구현되어 있으니 nn.Sigmoid()를 거치게되면 로지스틱 회귀의 가설식이다

In [53]:
# 훈련 데이터
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [54]:
# nn.Sequential()은 nn.Module 층을 차례로 쌓을 수 있도록한다
# 이를 이용해 인공 신경망 등을 구성하는데 사용된다.
model = nn.Sequential(
   nn.Linear(2, 1), # input_dim = 2, output_dim = 1
   nn.Sigmoid() # 출력은 시그모이드 함수를 거친다
)

In [55]:
# 예측값 확인, 가중치와 편향은 랜덤하게 초기화 되어 있음
print(list(model.parameters()))
print(model(x_train))

[Parameter containing:
tensor([[ 0.3643, -0.3121]], requires_grad=True), Parameter containing:
tensor([-0.1371], requires_grad=True)]
tensor([[0.4020],
        [0.4147],
        [0.6556],
        [0.5948],
        [0.6788],
        [0.8061]], grad_fn=<SigmoidBackward0>)


In [56]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

In [57]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 50번마다 로그 출력
    if epoch % 50 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f} Accuracy {accuracy*100:2.2f}%')

Epoch    0/1000 Cost: 0.539713 Accuracy 83.33%
Epoch   50/1000 Cost: 0.266094 Accuracy 83.33%
Epoch  100/1000 Cost: 0.134272 Accuracy 100.00%
Epoch  150/1000 Cost: 0.100483 Accuracy 100.00%
Epoch  200/1000 Cost: 0.080486 Accuracy 100.00%
Epoch  250/1000 Cost: 0.067248 Accuracy 100.00%
Epoch  300/1000 Cost: 0.057820 Accuracy 100.00%
Epoch  350/1000 Cost: 0.050753 Accuracy 100.00%
Epoch  400/1000 Cost: 0.045251 Accuracy 100.00%
Epoch  450/1000 Cost: 0.040843 Accuracy 100.00%
Epoch  500/1000 Cost: 0.037228 Accuracy 100.00%
Epoch  550/1000 Cost: 0.034210 Accuracy 100.00%
Epoch  600/1000 Cost: 0.031649 Accuracy 100.00%
Epoch  650/1000 Cost: 0.029449 Accuracy 100.00%
Epoch  700/1000 Cost: 0.027538 Accuracy 100.00%
Epoch  750/1000 Cost: 0.025862 Accuracy 100.00%
Epoch  800/1000 Cost: 0.024381 Accuracy 100.00%
Epoch  850/1000 Cost: 0.023061 Accuracy 100.00%
Epoch  900/1000 Cost: 0.021877 Accuracy 100.00%
Epoch  950/1000 Cost: 0.020810 Accuracy 100.00%
Epoch 1000/1000 Cost: 0.019843 Accuracy 10

In [58]:
print(list(model.parameters()))

[Parameter containing:
tensor([[3.2534, 1.5181]], requires_grad=True), Parameter containing:
tensor([-14.4839], requires_grad=True)]


로지스틱 회귀는 인공 신경망으로 표현 가능함

결과적으로 위의 로지스틱 회귀는 다음 식 처럼 표현 할 수 있음

$ H(x) = sigmoid(x_1w_1 + x_2w_2 + b) $

다만 sigmoid function은 인공 신경망의 hidden layer(은닉층)으로 거의 사용되지 않음

### 클래스 형태로 구현

In [59]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim=1, output_dim=1):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [60]:
model = BinaryClassifier(2, 1)

In [61]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

In [62]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 50번마다 로그 출력
    if epoch % 50 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f} Accuracy {accuracy*100:2.2f}%')

Epoch    0/1000 Cost: 0.614994 Accuracy 66.67%
Epoch   50/1000 Cost: 0.366382 Accuracy 83.33%
Epoch  100/1000 Cost: 0.140280 Accuracy 100.00%
Epoch  150/1000 Cost: 0.103760 Accuracy 100.00%
Epoch  200/1000 Cost: 0.082549 Accuracy 100.00%
Epoch  250/1000 Cost: 0.068668 Accuracy 100.00%
Epoch  300/1000 Cost: 0.058858 Accuracy 100.00%
Epoch  350/1000 Cost: 0.051546 Accuracy 100.00%
Epoch  400/1000 Cost: 0.045878 Accuracy 100.00%
Epoch  450/1000 Cost: 0.041351 Accuracy 100.00%
Epoch  500/1000 Cost: 0.037649 Accuracy 100.00%
Epoch  550/1000 Cost: 0.034563 Accuracy 100.00%
Epoch  600/1000 Cost: 0.031951 Accuracy 100.00%
Epoch  650/1000 Cost: 0.029710 Accuracy 100.00%
Epoch  700/1000 Cost: 0.027766 Accuracy 100.00%
Epoch  750/1000 Cost: 0.026063 Accuracy 100.00%
Epoch  800/1000 Cost: 0.024558 Accuracy 100.00%
Epoch  850/1000 Cost: 0.023219 Accuracy 100.00%
Epoch  900/1000 Cost: 0.022020 Accuracy 100.00%
Epoch  950/1000 Cost: 0.020939 Accuracy 100.00%
Epoch 1000/1000 Cost: 0.019960 Accuracy 10